In [75]:
import sys
sys.path.insert(0, '..')

from rrdp_tools.rrdp import parse_snapshot_or_delta, parse_notification_file

import pandas as pd
import dataclasses
import requests
import pathlib

In [76]:

# df = pd.DataFrame.from_dict([
#     dataclasses.asdict(elem) for elem in parse_snapshot_or_delta(snapshot)
# ])
def dataframe_from_file(file: pathlib.Path) -> pd.DataFrame:
    with file.open('r') as f:
        try:
            df = pd.DataFrame.from_dict([
                dataclasses.asdict(elem) for elem in parse_snapshot_or_delta(f)
            ])
    
            df['file'] = file
    
            return df
        except Exception as e:
            print(e)
            print(f"Error parsing {file}")
            return pd.DataFrame([])

def object_type(uri: str) -> str:
    return uri.split(".")[-1]

In [77]:
dfs = [dataframe_from_file(fn) for fn in pathlib.Path("../rrdp_content/files/").glob("*.xml")]

df = pd.concat(dfs)
#dataframe_from_file(next(iter(pathlib.Path("../rrdp_content/files/").glob("*.xml"))))
df['object_type'] = df.uri.map(object_type)

stats = df[['file', 'object_type', 'uri']].groupby(['file', 'object_type']).count().reset_index()


Error parsing ../rrdp_content/files/notification.xml


In [78]:
stats

,file,object_type,uri
0,../rrdp_content/files/35291.xml,crl,183
1,../rrdp_content/files/35291.xml,mft,183
2,../rrdp_content/files/35292.xml,crl,541
3,../rrdp_content/files/35292.xml,mft,541
4,../rrdp_content/files/35293.xml,crl,400
...,...,...,...
2352,../rrdp_content/files/39983.xml,mft,1
2353,../rrdp_content/files/39983.xml,roa,2
2354,../rrdp_content/files/39984.xml,crl,475
2355,../rrdp_content/files/39984.xml,mft,475


In [79]:
stats_wide = pd.pivot(stats, index='file', columns='object_type', values='uri').fillna(0)
stats_wide['user_action'] = (stats_wide['roa'] != 0) | (stats_wide['cer'] != 0)
stats_wide

object_type,cer,crl,mft,roa,user_action
file,,,,,
../rrdp_content/files/35291.xml,0.0,183.0,183.0,0.0,False
../rrdp_content/files/35292.xml,0.0,541.0,541.0,0.0,False
../rrdp_content/files/35293.xml,0.0,400.0,400.0,0.0,False
../rrdp_content/files/35294.xml,0.0,1.0,1.0,2.0,True
../rrdp_content/files/35295.xml,0.0,567.0,567.0,0.0,False
...,...,...,...,...,...
../rrdp_content/files/39980.xml,0.0,1.0,1.0,2.0,True
../rrdp_content/files/39981.xml,0.0,572.0,572.0,2.0,True
../rrdp_content/files/39982.xml,0.0,1.0,1.0,3.0,True


In [80]:
print(stats_wide.groupby(['user_action']).count().to_markdown())

| user_action   |   cer |   crl |   mft |   roa |
|:--------------|------:|------:|------:|------:|
| False         |   362 |   362 |   362 |   362 |
| True          |   533 |   533 |   533 |   533 |


# Now repeating these stats for `rrdp.paas.rpki.ripe.net`

In [81]:
dfs = [dataframe_from_file(fn) for fn in pathlib.Path("../rrdp_content/files-paas/").glob("*.xml")]

df = pd.concat(dfs)
#dataframe_from_file(next(iter(pathlib.Path("../rrdp_content/files/").glob("*.xml"))))
df['object_type'] = df.uri.map(object_type)

stats = df[['file', 'object_type', 'uri']].groupby(['file', 'object_type']).count().reset_index()


Error parsing ../rrdp_content/files-paas/notification.xml


In [82]:
stats_wide = pd.pivot(stats, index='file', columns='object_type', values='uri').fillna(0)
for type in ['cer', 'roa', 'mft', 'asa', 'gbr', 'roa']:
    if type not in stats_wide.keys():
        stats_wide[type] = 0

stats_wide['user_action'] = (stats_wide['roa'] != 0) | (stats_wide['cer'] != 0)
stats_wide

object_type,crl,mft,roa,cer,asa,gbr,user_action
file,,,,,,,
../rrdp_content/files-paas/18559.xml,1.0,1.0,0.0,0,0,0,False
../rrdp_content/files-paas/18560.xml,1.0,1.0,0.0,0,0,0,False
../rrdp_content/files-paas/18561.xml,1.0,1.0,0.0,0,0,0,False
../rrdp_content/files-paas/18562.xml,1.0,1.0,0.0,0,0,0,False
../rrdp_content/files-paas/18563.xml,1.0,1.0,0.0,0,0,0,False
...,...,...,...,...,...,...,...
../rrdp_content/files-paas/18624.xml,1.0,1.0,0.0,0,0,0,False
../rrdp_content/files-paas/18625.xml,1.0,1.0,0.0,0,0,0,False
../rrdp_content/files-paas/18626.xml,1.0,1.0,0.0,0,0,0,False


In [84]:
print(stats_wide.groupby(['user_action']).count().to_markdown())

| user_action   |   crl |   mft |   roa |   cer |   asa |   gbr |
|:--------------|------:|------:|------:|------:|------:|------:|
| False         |    63 |    63 |    63 |    63 |    63 |    63 |
| True          |     7 |     7 |     7 |     7 |     7 |     7 |
